In [55]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session



In [59]:
sql = """
select station_name, icscode, lat, lng from tt_h.all_stations 
where lat < 53.39926695813 
and lng > -2.54 and 
in_sprawl = false
"""

df = pd.read_sql(sql, conn)

In [61]:
# df["dest_pc"]= "OX145FP"
# df2 = df.copy()
# df2["dest_pc"] = "CB113QY"

# df21 = df.copy()
# df21["dest_pc"] = "CM25LB"

# df3 = pd.concat([df,df2,df3])
# num = len(df3)

In [4]:
df3["id"] = range(len(df3))

In [5]:
from google import get_drive_info

In [7]:
# Create new table of each station and drive time and distane 

# df3.to_sql("stations_drivetime", engine, schema="tt_h", if_exists="replace", index=False)

# sql = """

# ALTER TABLE tt_h.stations_drivetime  ADD PRIMARY KEY (id);

# ALTER TABLE tt_h.stations_drivetime ADD google_request text;
# UPDATE tt_h.stations_drivetime SET google_request = null;

# ALTER TABLE tt_h.stations_drivetime ADD google_response text;
# UPDATE tt_h.stations_drivetime SET google_response = null;

# ALTER TABLE tt_h.stations_drivetime ADD drive_minutes float;
# UPDATE tt_h.stations_drivetime SET drive_minutes = null;

# ALTER TABLE tt_h.stations_drivetime ADD drive_miles float;
# UPDATE tt_h.stations_drivetime SET drive_miles = null;


# """
# cursor.execute(sql)
# conn.commit()

In [47]:
# Then populate with drive times
sql = """
select id from tt_h.stations_drivetime where drive_minutes is null
"""
ids = pd.read_sql(sql, conn)
ids = [int(i) for i in ids["id"]]
len(ids)

0

In [46]:
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session
Stations = Automapped_Base.classes.stations_drivetime

counter = 0
for id in ids:
    station = session.query(Stations).filter(Stations.id == id).one()
    
    try:
        info = get_drive_info(origin_lat = station.lat , origin_lng = station.lng , dest_pc = station.dest_pc)
        station.google_request = info["request"]
        station.google_response = info["response"]
        station.drive_minutes = info["minutes"]
        station.drive_miles = info["miles"]
        session.add(station)
        session.commit()
    except Exception as e:
        print("error on {}".format(counter))
        break
    
    counter +=1
    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12


In [74]:
sql = """
select *
from tt_h.stations_drivetime
"""
df = pd.read_sql(sql, conn)
df.sample(20)

,station_name,icscode,lat,lng,dest_pc,id,google_request,google_response,drive_minutes,drive_miles
2647,Grindleford,90003125,53.305560,-1.626297,OX145FP,137,https://maps.googleapis.com/maps/api/distancem...,"{\n ""destination_addresses"" : [ ""Neave Mews,...",166.166667,151.126126
616,Bracknell,90000841,51.413090,-0.751704,OX145FP,462,https://maps.googleapis.com/maps/api/distancem...,"{\n ""destination_addresses"" : [ ""Neave Mews,...",53.733333,44.881627
2441,Camberley,90001594,51.336326,-0.744272,CM25LB,2411,None,None,NaN,NaN
1627,Stechford,90006802,52.484819,-1.811025,CM25LB,1966,None,None,NaN,NaN
2230,Little Kimble,1000528,51.752230,-0.808452,CB113QY,1950,https://maps.googleapis.com/maps/api/distancem...,"{\n ""destination_addresses"" : [ ""Saffron Wal...",83.016667,71.179291
2511,North Camp,90005303,51.275793,-0.731199,CB113QY,1288,https://maps.googleapis.com/maps/api/distancem...,"{\n ""destination_addresses"" : [ ""Saffron Wal...",98.683333,88.646030
2641,Flitwick,90002748,52.003640,-0.495258,CM25LB,2521,None,None,NaN,NaN
542,East Worthing,90002643,50.821643,-0.354894,OX145FP,350,https://maps.googleapis.com/maps/api/distancem...,"{\n ""destination_addresses"" : [ ""Neave Mews,...",124.566667,110.795420
413,Slade Green,1001252,51.467784,0.190491,CM25LB,2911,None,None,NaN,NaN
828,East Malling,90002569,51.285808,0.439280,OX145FP,760,https://maps.googleapis.com/maps/api/distancem...,"{\n ""destination_addresses"" : [ ""Neave Mews,...",113.266667,103.735403
